In [222]:
import pandas as pd
import numpy as np
from scipy import stats
import math
from pydataset import data
from env import get_db_url

### For each of the following questions, formulate a null and alternative hypothesis (be as specific as you can be), then give an example of what a true positive, true negative, type I and type II errors would look like.



- **Is the website redesign any good?**

>**H<sub>o</sub>:** website redesign is not good, click rate has no significant change
>
>>False Negative **(type II error)** Click rate has not changed, but another metric has improved that we're not tracking
>
>**H<sub>a</sub>:** website redesign is good, click rate has significantly improved
>
>>False Positive **(type I error)** Click rate has signifaicantly improved but not due to website redesign


- **Is our television ad driving more sales?**

>**H<sub>o</sub>:** Ad has no effect on sales, no significant increase
>
>>False Negative **(type II error)** sales have not risen, but due to other causes
>
>**H<sub>a</sub>:** Ad has boosted sales
>
>>False Positive **(type I error)** Sales have risen, but due to other causes

- **Has the network latency gone up since we switched internet service providers?**

>**H<sub>o</sub>:** Network latency has not been affected by new internet provider
>
>>False Negative **(type II error)** latency has risen but measurment device is not working
>
>**H<sub>a</sub>:** Network latency has risen significantly
>
>>False Positive **(type I error)** latency has not risen but measurment device is not working

# T-tests

- Ace Realty wants to determine whether the average time it takes to sell homes is different for its two offices. A sample of 40 sales from office #1 revealed a mean of 90 days and a standard deviation of 15 days. A sample of 50 sales from office #2 revealed a mean of 100 days and a standard deviation of 20 days. Use a .05 level of significance.

In [223]:
#metrics
dist_1 = stats.norm(90, 15).rvs(40)
dist_2 = stats.norm(100,20).rvs(50)

n1 = 40
n2 = 50

μ1 = 90
μ2 = 100

σ1 = 15
σ2 = 20

alpha = .05

In [224]:
#two sample t-test
#h0: time1 = time2
#ha: time1 ≠ time2
deg_f = n1 + n2 - 2
σ_pool = math.sqrt(((n1 - 1)*(σ1 ** 2) + (n2 - 1)*(σ2 ** 2)) / (deg_f))

t = (μ1 - μ2) / (σ_pool * math.sqrt(1/n1 + 1/n2))
t

-2.6252287036468456

In [225]:
p = stats.t(deg_f).cdf(t)
p

0.005104926224619695

In [226]:
p > alpha

False

### Load the mpg dataset and use it to answer the following questions:


In [227]:
mpg = data('mpg')

In [228]:
mpg['avg_mpg'] = (mpg.hwy+mpg.cty)/2

In [229]:
mpg.head(5)

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class,avg_mpg
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,23.5
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,25.0
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,25.5
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,25.5
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,21.0


In [230]:
volks_hwy_mpg = mpg[mpg['manufacturer'] == 'volkswagen'].hwy
volks_hwy_mpg
stats.ttest_1samp(volks_hwy_mpg, mpg.hwy.mean())

Ttest_1sampResult(statistic=5.652041311757083, pvalue=6.068818710469792e-06)

In [231]:
compact_mpg = mpg[mpg['class'] == 'compact'].cty
midsize_mpg = mpg[mpg['class'] == 'midsize'].cty
stats.ttest_ind(compact_mpg, midsize_mpg)

Ttest_indResult(statistic=2.2845195147123536, pvalue=0.024803276085898066)

- Is there a difference in fuel-efficiency in cars from 2008 vs 1999?


In [232]:
μ = mpg.groupby('year').avg_mpg.mean()
σ = mpg.groupby('year').avg_mpg.std()
n = mpg.groupby('year').avg_mpg.count()

In [233]:
#Ho: mu_2008 == mu_1999
#Ha: mu_2008 != mu_1999
deg_f = n[1999] + n[2008] - 2
σ_pool = math.sqrt(((μ[1999] - 1)*(σ[1999] ** 2) + (μ[2008] - 1)*(σ[2008] ** 2)) / (deg_f))

t = (μ[1999] - μ[2008]) / (σ_pool * math.sqrt(1/n[1999] + 1/n[2008]))
t

0.5404265072353494

In [234]:
p = stats.t(deg_f).cdf(t)
p

0.7052893195269174

- Are compact cars more fuel-efficient than the average car?

In [235]:
#ho: compact_mpg <= avg_mpg
#ha: compact_mpg >  avg_mpg

In [236]:
μ_all = mpg.avg_mpg.mean()
σ_all = mpg.avg_mpg.std()
n_all = mpg.avg_mpg.count()

μ_com = mpg[mpg['class'] == 'compact'].avg_mpg.mean()
σ_com = mpg[mpg['class'] == 'compact'].avg_mpg.std()
n_com = mpg[mpg['class'] == 'compact'].avg_mpg.count()


In [237]:
deg_f = n_com + n_all - 2
σ_pool = math.sqrt(((μ_com - 1)*(σ_com ** 2) + (μ_all - 1)*(σ_all ** 2)) / (deg_f))

t = (μ_com - μ_all) / (σ_pool * math.sqrt(1/n_com + 1/n_all))
t

15.22973882764796

In [238]:
p = stats.t(deg_f).cdf(t)
p

1.0

- Do manual cars get better gas mileage than automatic cars?

In [239]:
#Ho: standard mpg = automatic mpg
#Ha: standard mpg > automatic mpg
mpg['simple_trans'] = mpg.trans.str.startswith('manual')
mpg.head(5)

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class,avg_mpg,simple_trans
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,23.5,False
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,25.0,True
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,25.5,True
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,25.5,False
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,21.0,False


In [240]:
μ = mpg.groupby('simple_trans').avg_mpg.mean()
σ = mpg.groupby('simple_trans').avg_mpg.std()
n = mpg.groupby('simple_trans').avg_mpg.count()

In [241]:
deg_f = n[True] + n[False] - 2
σ_pool = math.sqrt(((μ[True] - 1)*(σ[False] ** 2) + (μ[True] - 1)*(σ[False] ** 2)) / (deg_f))

t = (μ[True] - μ[False]) / (σ_pool * math.sqrt(1/n[True] + 1/n[False]))
t

11.107655934792621

In [242]:
p = stats.t(deg_f).cdf(t)
p

1.0

# Correlation



### Class Example
 - is there a correlation between city mpg and displacement?
 - $H_o$ there is a correleation
 - $H_a$ there is no correlation

In [243]:
mpg = data('mpg')
stats.pearsonr(mpg['cty'], mpg['displ'])

(-0.7985239689348551, 4.737914890205637e-53)

### Use the telco_churn data.


In [244]:
#get db url
url = get_db_url('telco_churn')
telco = pd.read_sql('''
    SELECT customer_id, tenure, monthly_charges, total_charges, churn, phone_service as phone, internet_service_type_id as internet
    FROM customers''',
    url)
telco['internet'] = ((telco.internet == 1) | (telco.internet == 2))
telco['phone_and_internet'] = (telco.internet) & (telco.phone)
telco.sort_values('total_charges').tail(7032).head(5)

,customer_id,tenure,monthly_charges,total_charges,churn,phone,internet,phone_and_internet
4386,6180-YBIQI,5,24.30,100.2,No,No,True,True
2222,3178-FESZO,1,100.25,100.25,Yes,Yes,True,True
1771,2587-YNLES,6,20.10,100.35,No,Yes,False,False
5542,7802-EFKNY,5,24.95,100.4,Yes,No,True,True
5126,7216-EWTRS,1,100.80,100.8,Yes,Yes,True,True


- Does tenure correlate with monthly charges? 

In [245]:
x = telco.tenure
y = telco.monthly_charges
corr, p = stats.pearsonr(x,y)
{'correlation': corr, 'probability' : p}

{'correlation': 0.24789985628615002, 'probability': 4.0940449915016345e-99}

- Total charges?


In [246]:
no_null_telco = telco.sort_values('total_charges').tail(7032)
x = no_null_telco.tenure
y = no_null_telco.total_charges.apply(float)
corr, p = stats.pearsonr(x,y)
{'correlation': corr, 'probability' : p}

{'correlation': 0.825880460933202, 'probability': 0.0}

- What happens if you control for phone and internet service?

In [247]:
x = telco[telco['phone_and_internet'] == True].tenure
y = telco[telco['phone_and_internet'] == True].monthly_charges
corr, p = stats.pearsonr(x,y)
{'correlation': corr, 'probability' : p}

{'correlation': 0.3723066263198703, 'probability': 5.890963580482675e-181}

In [248]:
x = no_null_telco[no_null_telco['phone_and_internet'] == True].tenure
y = no_null_telco[no_null_telco['phone_and_internet'] == True].total_charges.apply(float)
corr, p = stats.pearsonr(x,y)
{'correlation': corr, 'probability' : p}

{'correlation': 0.9319346187418098, 'probability': 0.0}

### Use the employees database.

In [249]:
url = get_db_url('employees')
emp_sal = pd.read_sql(
'''
SELECT e.emp_no, DATEDIFF(now(), e.hire_date) as tenure, s.salary
FROM employees as e
JOIN salaries as s
USING(emp_no)
WHERE to_date > now()
''', url)
emp_sal.head(5)

,emp_no,tenure,salary
0,10001,12156,88958
1,10002,12373,72527
2,10003,12093,43311
3,10004,11998,74057
4,10005,10982,94692


- Is there a relationship between how long an employee has been with the company and their salary?


In [250]:
x = emp_sal.tenure
y = emp_sal.salary
corr, p = stats.pearsonr(x,y)
{'correlation': corr, 'probability' : p}

{'correlation': 0.30646256131860783, 'probability': 0.0}

- Is there a relationship between how long an employee has been with the company and the number of titles they have had?

In [261]:
emp_titles = pd.read_sql('SELECT emp_no, title FROM titles', url)
emp_titles = emp_titles.groupby('emp_no').count().head(5)

In [265]:
df = pd.merge(emp_sal, emp_titles, how = 'inner', on = 'emp_no')
df.head(5)

,emp_no,tenure,salary,title
0,10001,12156,88958,1
1,10002,12373,72527,1
2,10003,12093,43311,1
3,10004,11998,74057,2
4,10005,10982,94692,2


In [266]:
stats.pearsonr(df.title, df.salary)

(0.44208391033620664, 0.45603498012258536)

### Use the sleepstudy data. Is there a relationship between days and reaction time?

- $H_o$ Days correlate with reaction time
- $H_a$ There is no apparent correlation

In [268]:
ssd = data('sleepstudy')
ssd.head(5)

,Reaction,Days,Subject
1,249.5600,0,308
2,258.7047,1,308
3,250.8006,2,308
4,321.4398,3,308
5,356.8519,4,308


In [269]:
stats.pearsonr(ssd.Reaction, ssd.Days)

(0.5352302262650253, 9.894096322214812e-15)

# Chi Squared



### Use the following contingency table to help answer the question of whether using a macbook and being a codeup student are independent of each other.

In [316]:
index = ['macbook', 'no_Macbook']
columns = ['student', 'not_student']

observed = pd.DataFrame([[49, 20],[1, 30]], index = index, columns = columns)
observed

,student,not_student
macbook,49,20
no_Macbook,1,30


In [318]:
chi2, p, degf, expected = stats.chi2_contingency(observed)

chi2, p

(36.65264142122487, 1.4116760526193828e-09)

### Choose another 2 categorical variables from the mpg dataset and perform a chi2 contingency table test with them. Be sure to state your null and alternative hypotheses.

- $H_o:$ Manufacturer has no effect on prevalence of transmissions
- $H_a:$ Volkswagens manufacture more manual transmission vehicles than Audi

In [345]:
mpg.head(5)

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact


In [342]:
audi_auto = mpg[(mpg.manufacturer == 'audi')&(mpg.trans.str.startswith('auto'))].trans.count()
audi_manual = mpg[(mpg.manufacturer == 'audi')&(mpg.trans.str.startswith('manual'))].trans.count()
volkswagen_auto = mpg[(mpg.manufacturer == 'volkswagen')&(mpg.trans.str.startswith('auto'))].trans.count()
volkswagen_manual = mpg[(mpg.manufacturer == 'volkswagen')&(mpg.trans.str.startswith('manual'))].trans.count()

observed = pd.DataFrame([[audi_auto, audi_manual], [volkswagen_auto,volkswagen_manual]], columns = ['auto', 'manual'], index = ['audi', 'volkswagen'])
observed

,auto,manual
audi,11,7
volkswagen,13,14


In [344]:
stats.chi2_contingency(observed)

(0.3013392857142859, 0.5830440246512352, 1, array([[ 9.6,  8.4],
        [14.4, 12.6]]))

- $H_o:$ Transmission has no effect on mileage
- $H_a:$ maunal transmissions have higher mileages

In [355]:
df = mpg[['trans', 'cty', 'hwy']]
df['avg_mpg'] = (df.cty+df.hwy)/2
df['good_mpg'] = df.avg_mpg >= 25

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [368]:
good_auto = df[df.trans.str.startswith('auto')& df.good_mpg].trans.count()
good_manual = df[df.trans.str.startswith('manual')& df.good_mpg].trans.count()
bad_auto = df[df.trans.str.startswith('auto')& ~df.good_mpg].trans.count()
bad_manual = df[df.trans.str.startswith('manual')& ~df.good_mpg].trans.count()

observed = pd.DataFrame([[good_auto, good_manual], [bad_auto, bad_manual]], index = ['good_mpg','bad_mpg'], columns = ['auto', 'manual'])
observed

,auto,manual
good_mpg,17,26
bad_mpg,140,51


In [369]:
chi2, p, degf, expected = stats.chi2_contingency(observed)

chi2, p

(16.625694591485967, 4.553001057526927e-05)